# Train a Land Cover Classification Model

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_landcover_classification.ipynb)

## Install packages

To use the new functionality, ensure the required packages are installed.

In [ ]:
# %pip install geoai-py

## Import libraries

In [ ]:
import geoai

## Download sample data

In [ ]:
train_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/m_3807511_ne_18_060_20181104.tif"
train_landcover_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/m_3807511_ne_18_060_20181104_landcover.tif"
test_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/m_3807511_se_18_060_20181104.tif"

In [ ]:
train_raster_path = geoai.download_file(train_raster_url)
train_landcover_path = geoai.download_file(train_landcover_url)
test_raster_path = geoai.download_file(test_raster_url)

## Visualize sample data

In [ ]:
geoai.view_raster(train_landcover_url, basemap=train_raster_url)

In [ ]:
geoai.view_raster(test_raster_url)

## Create training data

We will use the NAIP dataset for land cover classification. The classification scheme is adopted from the [Chesapeake Land Cover](https://planetarycomputer.microsoft.com/dataset/chesapeake-lc-13) project.

**Important Note for Land Cover Classification:**
- Your label images should contain integer class values (0, 1, 2, ..., 13 for 13 classes)
- Do NOT use binary masks - the training code now properly handles multi-class labels
- Class 0 is typically background, classes 1-12 are your land cover types

In [ ]:
out_folder = "landcover"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_landcover_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

## Train semantic segmentation model

Now we'll train a semantic segmentation model using the new `train_object_detection` function. This function supports various architectures from `segmentation-models-pytorch`:

- **Architectures**: `unet`, `deeplabv3`, `deeplabv3plus`, `fpn`, `pspnet`, `linknet`, `manet`
- **Encoders**: `resnet34`, `resnet50`, `efficientnet-b0`, `mobilenet_v2`, etc.

In [ ]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=4,
    num_classes=13,
    batch_size=8,
    num_epochs=50,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
    plot_curves=True,
)

## Run inference

Now we'll use the trained model to make predictions on the test image.

In [ ]:
# Define paths
masks_path = "naip_test_semantic_prediction.tif"
model_path = f"{out_folder}/unet_models/best_model.pth"

In [ ]:
# Run semantic segmentation inference
geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=4,
    num_classes=13,
    window_size=512,
    overlap=256,
    batch_size=4,
)

## Visualize results

In [ ]:
geoai.write_colormap(masks_path, train_landcover_path, output=masks_path)

In [ ]:
geoai.view_raster(masks_path, basemap=test_raster_url)